### Mode loading for pyqubo

In [27]:
from pyqubo import Binary, solve_qubo
import dimod
import time

### Mode loading for qiskit
#### - optimization tool and algorithm

In [2]:
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import (
    GroverOptimizer, MinimumEigenOptimizer, ADMMOptimizer)

from docplex.mp.model import Model 

from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import (
    QAOA, VQE, NumPyMinimumEigensolver)

#### - ibmq import

In [3]:
from qiskit import *
from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview
from qiskit.providers.ibmq import least_busy
from qiskit import Aer

In [4]:
IBMQ.load_account()

<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [5]:
provider = IBMQ.get_provider(hub = 'ibm-q', group = 'open', project = 'main')

In [6]:
provider=IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend_overview()

ibm_washington               ibm_perth                    ibm_auckland
--------------               ---------                    ------------
Num. Qubits:  127            Num. Qubits:  7              Num. Qubits:  27
Pending Jobs: 41             Pending Jobs: 7              Pending Jobs: 92
Least busy:   False          Least busy:   False          Least busy:   False
Operational:  True           Operational:  True           Operational:  True
Avg. T1:      103.2          Avg. T1:      163.8          Avg. T1:      158.9
Avg. T2:      97.3           Avg. T2:      125.3          Avg. T2:      137.5



ibm_cairo                    ibm_lagos                    ibm_hanoi
---------                    ---------                    ---------
Num. Qubits:  27             Num. Qubits:  7              Num. Qubits:  27
Pending Jobs: 190            Pending Jobs: 74             Pending Jobs: 94
Least busy:   False          Least busy:   False          Least busy:   False
Operational:  True           O

### Question 1

$\text{max}\ \ xy +x + y$

#### pyqubo - 

In [9]:
from pyqubo import Binary

x, y = Binary('x'), Binary('y')
eq1 = -(x*y + x + y)
m1_pyqubo = eq1.compile()
qubo1, offset = m1_pyqubo.to_qubo()
qubo1, offset

({('x', 'x'): -1.0, ('x', 'y'): -1.0, ('y', 'y'): -1.0}, 0.0)

##### Simulated Annealing: simulator
##### ExactSolver: simulator

In [10]:
sampleset11 = dimod.SimulatedAnnealingSampler().sample_qubo(qubo1)
print(sampleset11)

   x  y energy num_oc.
0  1  1   -3.0       1
1  1  1   -3.0       1
2  1  1   -3.0       1
3  1  1   -3.0       1
4  1  1   -3.0       1
5  1  1   -3.0       1
6  1  1   -3.0       1
7  1  1   -3.0       1
9  1  1   -3.0       1
8  0  1   -1.0       1
['BINARY', 10 rows, 10 samples, 2 variables]


In [11]:
sampleset12 = dimod.ExactSolver().sample_qubo(qubo1)
print(sampleset12)

   x  y energy num_oc.
2  1  1   -3.0       1
1  1  0   -1.0       1
3  0  1   -1.0       1
0  0  0    0.0       1
['BINARY', 4 rows, 4 samples, 2 variables]


#### qiskit_simulator - 


In [12]:
m1_qiskit = Model('docplex_model')
x, y = m1_qiskit.binary_var_list(2)
m1_qiskit.maximize(x*y + x + y)

qp1 = QuadraticProgram()
qp1.from_docplex(m1_qiskit)

In [26]:
qp1

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model

Maximize
 obj: x0 + x1 + [ 2 x0*x1 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1

Binaries
 x0 x1
End

##### GroverOptimizer: statevector_simulator

In [13]:
# use the Grover Adaptive Search (GAS) to solve the optimization problem
start = time.time()

result103 =  GroverOptimizer(
    num_value_qubits = 3,
    quantum_instance =  Aer.get_backend('statevector_simulator')
    ).solve(qp1)

end = time.time()
result103, ('running time:', end - start)

(optimal function value: 3.0
 optimal value: [1. 1.]
 status: SUCCESS,
 ('running time:', 0.5053675174713135))

##### MinimumEigenOptimizer: 
 QAOA(qasm_simulator), QAOA(statevector_simulator) <br>
 VQE(qasm_simulator), VQE(statevector_simulator)<br>
 NumPyMinimumEigensolver()

In [14]:
start = time.time()

result_111 = MinimumEigenOptimizer(
    QAOA(quantum_instance = Aer.get_backend('qasm_simulator'))
    ).solve(qp1)

end = time.time()
result_111, ('running time:', end - start)

(optimal function value: 3.0
 optimal value: [1. 1.]
 status: SUCCESS,
 ('running time:', 0.5200490951538086))

In [16]:
start = time.time()

result_121 = MinimumEigenOptimizer(
    VQE(quantum_instance = Aer.get_backend('qasm_simulator'))
    ).solve(qp1)

end = time.time()
result_121, ('running time:', end - start)

/home/nccu/anaconda3/lib/python3.9/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


(optimal function value: 3.0
 optimal value: [1. 1.]
 status: SUCCESS,
 ('running time:', 0.19124150276184082))

In [18]:
start = time.time()

result_131 = MinimumEigenOptimizer(
    NumPyMinimumEigensolver()
    ).solve(qp1)

end = time.time()
result_131, ('running time:', end - start)

(optimal function value: 3.0
 optimal value: [1. 1.]
 status: SUCCESS,
 ('running time:', 0.006888628005981445))

##### ADMMOptimizer: 
GroverOptimizer: statevector_simulator, <br>
MinimumEigenOptimizer: QAOA(statevector_simulator)

In [19]:
start = time.time()

result_141 = ADMMOptimizer(MinimumEigenOptimizer(
    VQE(quantum_instance = Aer.get_backend('qasm_simulator'))
    )).solve(qp1)

end = time.time()
result_141, ('running time:', end - start)

(optimal function value: 0.0
 optimal value: [0. 0.]
 status: SUCCESS,
 ('running time:', 0.08823370933532715))

#### qiskit-QC
##### MinimumEigenOptimizer: VQE(ibmq)

### Question 2

$$
\text{max}\ \ 2n_1 + 2n_2 + 3n_3 + 3n_4 + 2n_5 - 2(n_1n_2 + n_1n_3 + n_2n_4 + n_3n_4 + n_3n_5 + n_4n_5))
$$
#### pyqubo

In [ ]:
n1, n2, n3, n4, n5 = Binary('n1'), Binary('n2'), Binary('n3'), Binary('n4'), Binary('n5')
eq2 = -(2*n1 + 2*n2 + 3*n3 + 3*n4 + 2*n5 - 2*(n1*n2 + n1*n3 + n2*n4 + n3*n4 + n3*n5 + n4*n5))
m2_pyqubo = eq2.compile()
qubo2, offset = m2_pyqubo.to_qubo()
qubo2, offset, eq2

({('n1', 'n2'): 2.0,
  ('n5', 'n3'): 2.0,
  ('n3', 'n3'): -3.0,
  ('n4', 'n2'): 2.0,
  ('n5', 'n4'): 2.0,
  ('n4', 'n3'): 2.0,
  ('n2', 'n2'): -2.0,
  ('n5', 'n5'): -2.0,
  ('n3', 'n1'): 2.0,
  ('n4', 'n4'): -3.0,
  ('n1', 'n1'): -2.0},
 0.0,
 ((Binary(n5)*Num(2.000000)+Binary(n4)*Num(3.000000)+Binary(n3)*Num(3.000000)+Binary(n1)*Num(2.000000)+Binary(n2)*Num(2.000000))+(Binary(n4)*Binary(n5)+Binary(n3)*Binary(n5)+Binary(n3)*Binary(n4)+Binary(n2)*Binary(n4)+Binary(n1)*Binary(n2)+Binary(n1)*Binary(n3))*Num(2.000000)*Num(-1.000000))*Num(-1.000000))

##### Simulated Annealing: simulator

In [ ]:
sampleset21 = dimod.SimulatedAnnealingSampler().sample_qubo(qubo2)
print(sampleset21)

  n1 n2 n3 n4 n5 energy num_oc.
0  1  0  0  1  1   -5.0       1
1  1  0  0  1  0   -5.0       1
2  0  1  1  0  0   -5.0       1
3  1  0  0  1  0   -5.0       1
4  0  1  1  0  1   -5.0       1
5  0  1  1  0  0   -5.0       1
6  1  0  0  1  1   -5.0       1
7  0  1  1  0  1   -5.0       1
8  1  0  0  1  0   -5.0       1
9  0  1  1  0  0   -5.0       1
['BINARY', 10 rows, 10 samples, 5 variables]


##### ExactSolver: simulator

In [ ]:
import numpy as np

#### qiskit-simulator

In [ ]:
m2_qiskit = Model('docplex_model')
n1, n2, n3, n4, n5 = m2_qiskit.binary_var_list(5)
m2_qiskit.maximize(
    (2*n1 + 2*n2 + 3*n3 + 3*n4 + 2*n5 
     - 2*(n1*n2 + n1*n3 + n2*n4 + n3*n4 + n3*n5 + n4*n5))
)

qp2 = QuadraticProgram()
qp2.from_docplex(m2_qiskit)
print(qp2.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model

Maximize
 obj: 2 x0 + 2 x1 + 3 x2 + 3 x3 + 2 x4 + [ - 4 x0*x1 - 4 x0*x2 - 4 x1*x3
      - 4 x2*x3 - 4 x2*x4 - 4 x3*x4 ]/2
Subject To

Bounds
 0 <= x0 <= 1
 0 <= x1 <= 1
 0 <= x2 <= 1
 0 <= x3 <= 1
 0 <= x4 <= 1

Binaries
 x0 x1 x2 x3 x4
End



##### GroverOptimizer: statevector_simulator

In [ ]:
start = time.time()
result206 =  GroverOptimizer(
    num_value_qubits = 3,
    quantum_instance =  Aer.get_backend('statevector_simulator')
    ).solve(qp2)
result206
end = time.time()
result206, ('running time:', end - start)

(optimal function value: 0.0
 optimal value: [1 1 1 1 1]
 status: SUCCESS,
 ('running time:', 2.863307237625122))

##### MinimumEigenOptimizer: QAOA(statevector_simulator)

In [ ]:
T = []
for i in range(50):
    start = time.time()
    result_211 = MinimumEigenOptimizer(
        QAOA(quantum_instance = Aer.get_backend('statevector_simulator'))
        ).solve(qp2)
    end = time.time()
    T.append(end - start)
    
result_211, ('ave. running time:', np.mean(T))

(optimal function value: 5.0
 optimal value: [0. 1. 1. 0. 0.]
 status: SUCCESS,
 ('ave. running time:', 0.15712875366210938))

##### MinimumEigenOptimizer: QAOA(qasm_simulator)

In [ ]:
T = []
for i in range(50):
    start = time.time()
    result_212 = MinimumEigenOptimizer(
        QAOA(quantum_instance = Aer.get_backend('qasm_simulator'))
        ).solve(qp2)
    end = time.time()
    T.append(end - start)
    
result_212, ('ave. running time:', np.mean(T))

(optimal function value: 5.0
 optimal value: [1. 0. 0. 1. 0.]
 status: SUCCESS,
 ('ave. running time:', 0.28749561309814453))

##### MinimumEigenOptimizer: VQE(statevector_simulator)

In [ ]:
T = []
for i in range(50):
    start = time.time()
    result_221 = MinimumEigenOptimizer(
        VQE(quantum_instance = Aer.get_backend('statevector_simulator'))
    ).solve(qp2)
    end = time.time()
    T.append(end - start)
    
result_221, ('ave. running time:', np.mean(T))

KeyboardInterrupt: 

##### MinimumEigenOptimizer: VQE(qasm_simulator)

In [ ]:
T = []
for i in range(50):
    start = time.time()
    result_222 = MinimumEigenOptimizer(
        VQE(quantum_instance = Aer.get_backend('qasm_simulator'))
    ).solve(qp2)
    end = time.time()
    T.append(end - start)
    
result_222, ('ave. running time:', np.mean(T))

KeyboardInterrupt: 

#### qiskit-QC
##### MinimumEigenOptimizer: VQE(ibmq_qasm_simulator)

In [ ]:
start = time.time()
result_221 = MinimumEigenOptimizer(
    VQE(quantum_instance = QuantumInstance(
        skip_qobj_validation = False,
        backend = provider.get_backend('ibmq_qasm_simulator'))
    )).solve(qp2)
end = time.time()
result_221, ('running time:', end - start)

In [ ]:
start = time.time()
result_221 = MinimumEigenOptimizer(
    QAOA(quantum_instance = QuantumInstance(
        skip_qobj_validation = False,
        backend = provider.get_backend('ibmq_qasm_simulator'))
    )).solve(qp2)
end = time.time()
result_221, ('running time:', end - start)

##### MinimumEigenOptimizer: VQE(ibmq_16_melbourne)

In [ ]:
start = time.time()
result_222 = MinimumEigenOptimizer(
    VQE(quantum_instance = QuantumInstance(
        skip_qobj_validation = False,
        backend = provider.get_backend('ibmq_16_melbourne'))
    )).solve(qp2)
end = time.time()
result_222, ('running time:', end - start)

KeyboardInterrupt: 

In [ ]:
start = time.time()
result_222 = MinimumEigenOptimizer(
    QAOA(quantum_instance = QuantumInstance(
        skip_qobj_validation = False,
        backend = provider.get_backend('ibmq_16_melbourne'))
    )).solve(qp2)
end = time.time()
result_222, ('running time:', end - start)

### Question 3 - Min Cost Problem

$\text{min}$
$$
41x_{ab}+34x_{ac}+20x_{ba}+18x_{bc}+27x_{ca}+36x_{cb}
$$
$\text{subject to}$
$$
x_{ab}+x_{ac}-x_{ba}-x_{ca} = 400 \\
x_{ba}+x_{bc}-x_{ab}-x_{cb} = 0 \\
x_{ca}+x_{cb}-x_{ac}-x_{bc} = -400 \\
0 \leq x_{ab} \leq 140 \\
0 \leq x_{ac} \leq 170 \\
0 \leq x_{ba} \leq 124 \\
0 \leq x_{bc} \leq 100 \\
0 \leq x_{ca} \leq 142 \\
0 \leq x_{cb} \leq 180 \\
$$

#### qiskit-simulator

In [ ]:
qp3 = QuadraticProgram()
qp3.integer_var(lowerbound=0, upperbound=140, name='xab')
qp3.integer_var(lowerbound=0, upperbound=170, name='xac')
qp3.integer_var(lowerbound=0, upperbound=124, name='xba')
qp3.integer_var(lowerbound=0, upperbound=100, name='xbc')
qp3.integer_var(lowerbound=0, upperbound=142, name='xca')
qp3.integer_var(lowerbound=0, upperbound=180, name='xcb')

qp3.minimize(linear={'xab': 41, 'xac': 34,'xba': 20, 'xbc': 18, 'xca': 27, 'xcb': 36,})
qp3.linear_constraint(
    linear={'xab': 1, 'xac': 1, 'xca': -1, 'xcb': -1},
    sense='E', rhs=100, name='a_out')
qp3.linear_constraint(
    linear={'xba': 1, 'xbc': 1, 'xab': -1, 'xcb': -1},
    sense='E', rhs=0, name='allpath')
qp3.linear_constraint(
    linear={'xca': 1, 'xcb': 1, 'xac': -1, 'xbc': -1},
    sense='E', rhs=-100, name='c_in')

print(qp3.export_as_lp_string())

In [ ]:
from qiskit.optimization.converters import IntegerToBinary

qp3_bin = IntegerToBinary().convert(qp3)
print(qp3_bin.export_as_lp_string())

In [ ]:
from qiskit.optimization.converters import LinearEqualityToPenalty

qp3_qubo = LinearEqualityToPenalty().convert(qp3_bin)
print(qp3_qubo.export_as_lp_string())

In [ ]:
result_312 = MinimumEigenOptimizer(
    VQE(quantum_instance = Aer.get_backend('qasm_simulator'))
    ).solve(qp3_qubo)
result_312

In [ ]:
result_311 = MinimumEigenOptimizer(
    QAOA(quantum_instance = Aer.get_backend('statevector_simulator'))
    ).solve(qp3_qubo)
result_311

In [ ]:
result_313 = MinimumEigenOptimizer(
    VQE(quantum_instance = Aer.get_backend('qasm_simulator'))
    ).solve(qp3)
result_313

In [ ]:
result_314 = MinimumEigenOptimizer(
    VQE(quantum_instance = Aer.get_backend('statevector_simulator'))
    ).solve(qp3)
result_314